In [ ]:
'''
def dataset_spec(datasetfile,dataset_type,goal_standard,scale = 1,scale_fun="mean",outputdata = True,outfilename='out') 
##################by RootChen#######################
##this function is used for preparation of some open datasets 
# 
1、datasetfile (type:string)                                                      #*.txt (supoort for sensorscope and xieye_sysytem dataset) 数据集文件名
2、datatype:  (type:string)                                                      #'sensorscope' 、‘xi_eye’                                                 数据集类型
##
3、goal_standrd: (type:string)                                                 #goal data specification standard 目标标准
#3.1  “time series”                                                                #for arima,etc. time serial use
#3.2  “llibsvm”                                                                     #standard for libsvm (svm,svdd) +1 1:0.708333 2:1 3:1 4:-0.320755 
#3.3  etc
##
4、scale : (positive integer)                                                    #respample scale.   重采样间隔
#4.1   1 (default)                                                                    #means that uses the original dataset. 
#4.2  n                                                                                #set the scale = n , the dataset after specification will be reduced to nth of the original.
##
5、scale_fun : (type:string)                                                     #method of scale the dateset               
#5.1 mean                                                                            #mean value of  sliding window n 平均数
#5.2 median                                                                         #median value of  sliding window n中位数
#5.3 other data processing methods
##
6、outputdata: (type:bool)                                                      # whether output the data to txt
#6.1 True (default)
#6.2 False
##
7、outfilename：(type:string)                                                  #output file name if need any file name requirments
#out (default)

'''

In [1]:
#导入库文件
import numpy as np
import math
import matplotlib.pyplot as plt #引入绘图库
import time
import pandas as pd  
import datetime
import os
import sys

In [2]:
#定义函数

#本函数用于返回  存储的各数据集用于pandasdataframe 对于的数据属性项
def get_pd_attribute(dataset_type):
    if dataset_type.strip() =="":
        print("dataset_type is empty")
    elif dataset_type == "sensorscope":
        pd_type =  ['ID','Year','Month','Day','Hour','Minute','Second',\
                   'TimeStamp','Sequence Number','Ambient Temperature',\
                    'Surface Temperature','Solar Radiation','Relative Humidity',\
                    'Soil Moisture','Watermark','Rain Meter','Wind Speed',\
                     'Wind Direction']
        return pd_type
    elif dataset_type == "xi_eye":
        pd_type =  ['sequence','date','time','am','dest','src','length','groups','type_serial','options',\
                   'thl','etx','origin','originSeqNo','type_x','magic','type','source','sink','source_time',\
                    'sink_time','seqno','period','parent','metric','power','task_time','radio_time','forwarder',\
                    'Humidity',\
                    'Temperature','Photo_active',\
                    'reserved16','reserved32','mark','hopcount','pathlen',\
                     'node_1','node_2','node_3','node_4','node_5','node_6','node_7','node_8','node_9','node_10']
        print(len(pd_type))
        return pd_type
    else:
        return 
#数据流的关闭函数
def read_dataset_close(data_object):
    data_object.close()     

#read_dataset_file文件流方式读取数据集
def read_dataset_file(datefile,dataset_type):
    if datefile.strip() == '':
        print('no filename input')
    else:
        #打开文件
        pd_type = get_pd_attribute(dataset_type)
        if pd_type is None:
            print('pd_type is null,please write the code in function get_pd_attribute')
            return
        else:
            try:
                ###读取txt 文件
                data = pd.read_csv(datefile,names = pd_type,sep=' ')
                return data
            except IOError:
                print('文件读取异常！,请输入正确的文件名，或者查看文件目录是否正确')
                return
#根据数据集的类型提炼整合数据集中有效数据          
def data_refine(data,dataset_type,outfile):  #返回子数据集，存储文件到本地文件夹内
    if data is None:
        print('数据集为空')
    else:
        ############   sensorscope     ############
        if dataset_type =='sensorscope':
            print('this is sensorscope')
            df = pd.DataFrame({'year':data['Year'],'month':data['Month'],
                   'day':data['Day'],'hour':data['Hour'],'minute':data['Minute'],'second':data['Second']})
            df = pd.to_datetime(df)
            #插入日期数据
            data.insert(1,'DateTime',df)
            #设置日期列为索引
            #data.set_index('DateTime',inplace=True)
            
            ##***#可修改处，根据获取数据调整，按索引获取。
            #subdata = data.get(['DateTime','Ambient Temperature','Surface Temperature','Solar Radiation','Relative Humidity'])
            #subdata = data.get(['DateTime','Ambient Temperature','Relative Humidity'])
            subdata = data.get(['DateTime','Ambient Temperature'])
            subdata.set_index('DateTime',inplace=True)
            time = datetime.datetime.now() 
            time_str = datetime.datetime.strftime(time,'%Y-%m-%d %H:%M:%S')
            dirpath = './'+dataset_type+'_'+time_str
            if not os.path.exists(dirpath):  #如果不存在该文件夹，则创建
                os.mkdir(dirpath)
                print(dirpath+'文件夹创建成果')
            else:
                print('文件存在无需创建')
                
            path =  dirpath+'/'+outfile
            #subdata.to_csv(path,sep=' ',header = False,index=False)    
            subdata['Ambient Temperature'].to_csv(path)  
            return subdata
            #将nan项填充0
            
        ############   xi_eye     ############
        elif dataset_type =='xi_eye':     #使用数据分类后的值
            #data['DateTime'] = data['date']+' '+data['time']
            #设置日期列为索引
            #data.set_index('DateTime',inplace=True)
            ###可修改处，根据获取数据调整，按索引获取。
            subdata = data.get(['date','time','origin','Temperature','Humidity','Photo_active'])
            #dataframe按照ID数据重新排序
            subdata=subdata.sort_values(by="origin")
            subdata.drop_duplicates(subset=['date','time','origin'],keep='first',inplace=True)  #inplace = True 表示删除样本，false 表示将重复的部分另存一下副本
            #按照ID输出到txt文件中  
            #获取当前系统时间，以时间作名字，创建文件夹
            time = datetime.datetime.now() 
            time_str = datetime.datetime.strftime(time,'%Y-%m-%d %H:%M:%S')
            print(time_str)
            dirpath = './'+dataset_type+'_'+time_str   #文件夹名字 时间+数据集类型
            if not os.path.exists(dirpath):  #如果不存在该文件夹，则创建
                os.mkdir(dirpath)
                print(dirpath+'文件夹创建成果')
            else:
                print('文件存在无需创建')
    
            for i in range(100,131):
                #print(data_object[data_object['origin']==i])
                if not subdata[data_object['origin']==i].empty :
                    
                    df = subdata[data_object['origin']==i]
                    #print(df)
                    #time = datetime.datetime.now() 
                    #time_str = datetime.datetime.strftime(time,'%Y-%m-%d %H:%M:%S')
                    path = dirpath+'/'+outfile+str(i)
                    #outfilename = 'out'+str(i)+' '+time_str+'.txt'
                    df.to_csv(path,sep=' ',header = True,index=False)
                    
            #print(subdata)
            return subdata
        else:
            print('请在get_pd_attribute、data_refine函数中定义新的数据集 **%s **的数据属性和数据提炼方法',dataset_type)
            return 
        
        
#data resample  #参数dataframe  scale 比例 scalefun 算法函数 返回dataframe
#dataframe设计到时间的项，处理事将时间项和其他变量分开处理，时间项按比例的最后时间为准，其他项则进行处理。
def dataresample(data,scale,scalefun='interval'):  # 1、interval sample  2、mean resample   3、median resample  4、protoco
    #print('data的index',data.shape)
    #print('data的column',data.columns)
    #判断data是否为空
    if not data.empty:
        if scale>=1:  #判断数据重采样比例是否正常>=1
            if scalefun=='interval':
                rows = data.shape[0]
                index = np.arange(0,rows,1)
                #select_index = index[index%scale==0]
                select_index=index[index%scale==scale-1]   #取间隔最后一个值作为时间索引
                subdata = data.loc[select_index]
                #print(subdata)
                return subdata
            elif scalefun=='mean':
                #新建dataframe对象，每十个数据行求平均，加入到dataframe中
                rows = data.shape[0]
                #index = np.arange(0,rows,1)
                internal = scale
                #temp = scale-1
                 #subdata=data_object.loc[select_index]
                #subdata
                #由于求平均操作时间值不可以计算，所以提取数据中的时间列 和数据列。
                timedf = data[['DateTime']]
                #print(timedf)
                timedf = dataresample(timedf,internal,scalefun='interval')
                timedf = timedf.reset_index(drop=True)
                #print(timedf)
                #datadf = data[['Ambient Temperature','Surface Temperature','Solar Radiation','Relative Humidity']]
                #####可修改处
                #datadf = data[['Ambient Temperature','Relative Humidity']]
                datadf = data[['Ambient Temperature']]
               
                newdataframe = pd.DataFrame(columns=datadf.columns)
                #按照间隔求数值部分平均值
                for i  in range(rows//internal+1):
                    begin = i*internal
                    end = begin+internal
                    if end >rows:
                        end = rows
                    newdataframe.loc[i] = datadf[begin:end].apply(lambda x: x.mean())   #求区间的平均值
                #print(newdataframe)
                #result = pd.merge(timedf,newdataframe,left_index=True,right_index=True)
                result = timedf.join(newdataframe)
                return result
                    
            elif scalefun=='median':
                pass
            else:
                print('请在dataresample(data,scale,scalefun=interval)函数中添加相应重采样代码')
                pass
                
        else:      #缩减返回值<1.显示异常
            print('dataresample(data,scale,scalefun=interval).  scale 值为空')
            return 
    else:# 文件打开失败
        print('dataresample(data,scale,scalefun=interval).  data 值为空')
        return

##转化格式 libsvm
def format_data_forlibsvm(data,labels,outfile):  #输入dataframe变量、标签变量(训练集、测试集预先设定)。输出data数据至文本
    #待完成
    #1、插入异常值 label = -1  short+10  noise 随机噪声。
    
    
    #规约数据集至[-1,1]或者[0,1], 返回每列的平均。
    
    pass

In [3]:
def reSample(df,datatype,interval):  #df包含时间、数值。 interval 代表间隔周期
    #data_refine函数中已经处理过时间数据
    #sub = df[['DateTime',datatype]]
    #sub['Date'].astype('datetime64')
    #sub['DateTime'] = pd.to_datetime(sub['DateTime'])
    #sub.set_index('DateTime',inplace=True)
    resampe_sub = df.resample(interval).mean()
    return resampe_sub

In [4]:
#data_object.head()

In [5]:
#test1 read dataset
#data_object=read_dataset_file('data_c1.txt','xi_eye')  #xi_eye
datafilename = 'sensorscope-meteo-3.txt'
data_object=read_dataset_file(datafilename,'sensorscope')  

In [6]:
##2数据提炼
#data_object=data_refine(data_object,'xi_eye','testtest')   #xi_eye
outputfile = datafilename+ '.csv'
data_object=data_refine(data_object,'sensorscope',outputfile)

this is sensorscope
./sensorscope_2019-01-10 10:54:56文件夹创建成果


In [ ]:
#获取子集

subtype1 = '10min'
subtype2 = '30min'
subtype3 = '1h'
subdata1 = reSample(data_object,'Ambient Temperature',subtype1)
subdata2 = reSample(data_object,'Ambient Temperature',subtype2)
subdata3 = reSample(data_object,'Ambient Temperature',subtype3)



In [ ]:
#subdata.head()
outfilename1 = datafilename +'_'+subtype1+ '.csv'
outfilename2 = datafilename +'_'+subtype2+ '.csv'
outfilename3 = datafilename +'_'+subtype3+ '.csv'
subdata1['Ambient Temperature'].to_csv(outfilename1)
subdata2['Ambient Temperature'].to_csv(outfilename2)
subdata3['Ambient Temperature'].to_csv(outfilename3)
#data_object.drop_duplicates(subset=['DateTime','origin'],keep='first',inplace=True)  #inplace = True 表示删除样本，false 表示将重复的部分另存一下副本
#data_object

In [ ]:
'''
for i in range(100,131):
    #print(data_object[data_object['origin']==i])
    if not data_object[data_object['origin']==i].empty :
        df = data_object[data_object['origin']==i]
        print(df)
        outfilename = 'out'+str(i)+'.txt'
        df.to_csv(outfilename,sep=' ',header = False,index=False)
        #print(data_object[data_object['origin']==i])
'''

In [ ]:
#test3 创建文件夹
'''
time1 = datetime.datetime.now() 
time1_str = datetime.datetime.strftime(time1,'%Y-%m-%d %H:%M:%S')
time1_str
os.mkdir(time1_str)
#path = os.path.realpath(__file__)
outfilename = 'out'
path = time1_str+'/'+outfilename
data_object.to_csv(path,sep=' ',header = False,index=False)
'''

In [ ]:
data_object.head()

In [ ]:
##test 4 resample
# 重采样
''' interval
dataresample(data_object,10)

rows = data_object.shape[0]
index = np.arange(0,rows,1)
select_index=index[index%10==0]
subdata=data_object.loc[select_index]
subdata
'''
#mean
'''
rows = data_object.shape[0]
index = np.arange(0,rows,1)
internal = 4
select_index=index[index%internal==0]
#subdata=data_object.loc[select_index]
#subdata

timedf = data_object[['DateTime']]
print(timedf)
timedf = dataresample(timedf,internal,scalefun='interval')
print(timedf)
datadf = data_object[['Ambient Temperature','Surface Temperature','Solar Radiation','Relative Humidity']]
newdataframe = pd.DataFrame(columns=datadf.columns)
index==0
for i  in range(datadf.shape[0]//internal+1):
    begin = i*internal
    end = begin+internal
    if end >datadf.shape[0]:
        end = datadf.shape[0]
    #subdata = data_object[begin:end].mean()
    #print(subdata)
    #print(subdata.shape)
    #newdataframe.append(subdata,ignore_index=True)
    newdataframe.loc[i] = datadf[begin:end].apply(lambda x: x.mean())
'''
#整合两个dataframe
#newdataframe
#timedf.merge(newdataframe, left_on='DateTime', right_on='Ambient Temperature', how='outer')
#newdataframe.reset_index(drop=True)
'''
print(newdataframe)
timedf=timedf.reset_index(drop=True)
print(timedf)
data_object=timedf.join(newdataframe)
#newdataframe['index'] = newdataframe.index.apply(lambda x: x)
''''''
l1 = range(newdataframe.shape[0])
print(l1)
index1 = pd.Series(l1)
#index1 = index1.astype('float64')
print(index1)
newdataframe['join']=index1
#newdataframe.insert(1,'join',index1)
l2 = range(timedf.shape[0])
index2 = pd.Series(l2)
timedf['join']=index2

print(newdataframe)
print(timedf)
'''
'''
#result=pd.merge(timedf,newdataframe)
'''
#result = pd.merge(timedf,newdataframe,left_index=True,right_index=True)
#result
data_object = dataresample(data_object,4,scalefun='mean')
data_object

In [ ]:
#for row in range(0,rows = data_object.shape[0]):
 #   print(data_object.iloc[row])

In [ ]:
#test 5  格式转化输出 t x t  dataframe
columns = data_object.shape[1]